画最终模型的框架图

In [11]:
from keras.layers import *
from keras.applications import resnet50, inception_v3, xception
from keras.utils.vis_utils import plot_model
from keras import Model
from keras.preprocessing import image
import numpy as np

In [6]:
model_resnet50 = resnet50.ResNet50(weights='imagenet', include_top=False)
model_inceptionV3 = inception_v3.InceptionV3(weights='imagenet', include_top=False)
model_xception = xception.Xception(weights='imagenet', include_top=False)

x_resnet50 = Input(shape=(224,224,3))
x_res = model_resnet50(x_resnet50)
x_res = GlobalAveragePooling2D()(x_res)

x_inceptionV3 = Input(shape=(299,299,3))
x_inc = model_inceptionV3(x_inceptionV3)
x_inc = GlobalAveragePooling2D()(x_inc)

x_xception = Input(shape=(299,299,3))
x_xce = model_xception(x_xception)
x_xce = GlobalAveragePooling2D()(x_xce)

x = Concatenate()([x_res, x_inc, x_xce])

x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[x_resnet50, x_inceptionV3, x_xception], outputs=x)

plot_model(model, show_shapes=False, to_file='fianl_model.png')

In [9]:
plot_model(model, show_shapes=True, to_file='fianl_model.png')

测试模型运行时间，虽然此模型没有被训练，但是参数个数与最终模型相同，运行时间也相同

In [13]:
data_299 = np.zeros((100,299,299,3))
data_224 = np.zeros((100,224,224,3))
for i in range(100):
    img = image.load_img('./train/cat.'+str(i)+'.jpg', target_size=(299,299))
    data_299[i] = image.img_to_array(img)
    
    img = image.load_img('./train/cat.'+str(i)+'.jpg', target_size=(224,224))
    data_224[i] = image.img_to_array(img)

In [14]:
from timeit import timeit

In [19]:
def complex_pred():
    pred = model.predict([resnet50.preprocess_input(data_224), inception_v3.preprocess_input(data_299),
                         xception.preprocess_input(data_299)])

In [23]:
t = timeit('complex_pred()','from __main__ import complex_pred', number=20)
print(t)

70.56376170399972
